# Modulo 3. Prueba técnica

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway, shapiro, kstest
from sklearn.linear_model import LinearRegression

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

### Primera Fase

In [2]:
#Abrir ficheros

def open_file(ruta):
    csv_apertura= pd.read_csv(ruta, index_col=0)
    csv_apertura.reset_index(inplace = True)
    display(csv_apertura.sample(5))
    return csv_apertura


In [3]:
#Exploración datos

def data_exploring(csv):
    forma = csv.shape
    print(f"La forma es {forma}")
    print("_______________")
    columnas = csv.columns
    print(f"Las columnas son {columnas}")
    print("_______________")
    nulos = csv.isna().sum().reset_index()
    print(f"Los nulos son:")
    display(nulos)
    print("_______________")
    duplicados = csv.duplicated().sum()
    print(f"Hay {duplicados} duplicados")
    print("_______________")
    tipo_dato = csv.dtypes.reset_index()
    print(f"Los datos son de tipo:")
    display(tipo_dato)
    


In [4]:
flight_activity = open_file("Files/Customer_Flight_Activity.csv")

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
399262,661321,2018,12,17,0,17,4794,479.0,0,0
346354,545087,2018,9,1,0,1,1638,163.0,0,0
382077,645280,2018,11,0,0,0,0,0.0,0,0
61403,669501,2017,4,0,0,0,0,0.0,0,0
14416,865737,2017,1,0,0,0,0,0.0,0,0


In [5]:
data_exploring(flight_activity)

La forma es (405624, 10)
_______________
Las columnas son Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')
_______________
Los nulos son:


,index,0
0,Loyalty Number,0
1,Year,0
2,Month,0
3,Flights Booked,0
4,Flights with Companions,0
5,Total Flights,0
6,Distance,0
7,Points Accumulated,0
8,Points Redeemed,0
9,Dollar Cost Points Redeemed,0


_______________
Hay 1864 duplicados
_______________
Los datos son de tipo:


,index,0
0,Loyalty Number,int64
1,Year,int64
2,Month,int64
3,Flights Booked,int64
4,Flights with Companions,int64
5,Total Flights,int64
6,Distance,int64
7,Points Accumulated,float64
8,Points Redeemed,int64
9,Dollar Cost Points Redeemed,int64


In [6]:
#Quitamos valores duplicados

flight_activity.drop_duplicates(inplace=True)

print(flight_activity.duplicated().sum())
print(flight_activity.shape)

0
(403760, 10)


In [7]:
#Renombramos las columnas

fa_names = ["loyalty_number", "year", "month", "flights_booked", "flights_with_companions", "total_flights", "distance", "points_accumulated", "points_redeemed", "dollar_cost_points_redeemed"]
flight_activity.columns = fa_names

flight_activity.sample(1)
  

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
246971,521702,2018,2,8,0,8,2568,276.48,0,0


In [8]:
loyalty_history= open_file("Files/Customer_Loyalty_History.csv")

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
10115,278719,Canada,Ontario,Sudbury,M5V 1G5,Male,College,NaN,Divorced,Star,2309.88,Standard,2012,10,NaN,NaN
4971,535436,Canada,Quebec,Montreal,H4G 3T4,Female,Doctor,267656.0,Married,Nova,3312.18,Standard,2016,3,NaN,NaN
9357,985707,Canada,Quebec,Montreal,H2T 9K8,Female,College,NaN,Single,Nova,16702.70,Standard,2018,5,NaN,NaN
8126,365596,Canada,Ontario,Toronto,M2M 6J7,Male,College,NaN,Single,Aurora,10442.45,2018 Promotion,2018,3,NaN,NaN
14424,473758,Canada,British Columbia,Victoria,V10 6T5,Male,Bachelor,48899.0,Married,Star,7078.95,Standard,2013,3,NaN,NaN


In [9]:
data_exploring(loyalty_history)

La forma es (16737, 16)
_______________
Las columnas son Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')
_______________
Los nulos son:


,index,0
0,Loyalty Number,0
1,Country,0
2,Province,0
3,City,0
4,Postal Code,0
5,Gender,0
6,Education,0
7,Salary,4238
8,Marital Status,0
9,Loyalty Card,0


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,Loyalty Number,int64
1,Country,object
2,Province,object
3,City,object
4,Postal Code,object
5,Gender,object
6,Education,object
7,Salary,float64
8,Marital Status,object
9,Loyalty Card,object


#### Evaluación Nulos

##### Columna Salary

In [12]:
#Evaluación de nulos

sal_null = np.round((loyalty_history["Salary"].isnull().sum() / loyalty_history.shape[0]) * 100)

print(f'el valor de los nulos en la columna salary representa un {sal_null}% de los datos')

print("......................................................................................")

null_education = loyalty_history.groupby('Education')['Salary'].apply(lambda x: x.isnull().sum())

print(null_education.reset_index())

print("......................................................................................")

salary_education = np.round(loyalty_history.groupby('Education')['Salary'].mean(),2)

print(salary_education.reset_index())

print("......................................................................................")

customer_education = loyalty_history['Education'].value_counts()

print(customer_education.reset_index())



el valor de los nulos en la columna salary representa un 25.0% de los datos
......................................................................................
              Education  Salary
0              Bachelor       0
1               College    4238
2                Doctor       0
3  High School or Below       0
4                Master       0
......................................................................................
              Education     Salary
0              Bachelor   72451.06
1               College        NaN
2                Doctor  178608.90
3  High School or Below   61071.72
4                Master  103757.85
......................................................................................
                  index  Education
0              Bachelor      10475
1               College       4238
2  High School or Below        782
3                Doctor        734
4                Master        508


El nivel educativo collage no tiene información en la columna salario por tanto, analizamos más en profundidad que ocurre en las demás columnas

In [13]:
lh_collage = loyalty_history.loc[loyalty_history["Education"]=="College",:]

lh_collage

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
6,927943,Canada,Ontario,Toronto,P5S 6R4,Female,College,NaN,Single,Star,3857.95,Standard,2014,6,NaN,NaN
13,988178,Canada,Quebec,Montreal,H4G 3T4,Male,College,NaN,Single,Star,3871.07,Standard,2013,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16721,632951,Canada,Alberta,Edmonton,T9G 1W3,Female,College,NaN,Married,Star,44771.30,Standard,2018,7,NaN,NaN
16727,546773,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Married,Star,52811.49,Standard,2015,9,NaN,NaN
16731,900501,Canada,Ontario,Sudbury,M5V 1G5,Male,College,NaN,Single,Star,61134.68,Standard,2012,9,NaN,NaN
16732,823768,Canada,British Columbia,Vancouver,V6E 3Z3,Female,College,NaN,Married,Star,61850.19,Standard,2012,12,NaN,NaN


En función de la evaluación, como el resto de columnas sí contienen datos, vamos a reemplazar los nulos por cero. De esta forma, podremos analizar el resto de información para el nivel educativo college, excepto para la variable salary

##### Columna cancellation_year	

In [15]:
#Evaluación de nulos

cany_null = np.round((loyalty_history["Cancellation Year"].isnull().sum() / loyalty_history.shape[0]) * 100)

print(f'el valor de los nulos en la columna cancellation_year representa un {cany_null}% de los datos')

el valor de los nulos en la columna cancellation_year representa un 88.0% de los datos


##### Columna cancellation_month

In [17]:
#Evaluación de nulos

canm_null = np.round((loyalty_history["Cancellation Month"].isnull().sum() / loyalty_history.shape[0]) * 100)

print(f'el valor de los nulos en la columna cancellation_month representa un {canm_null}% de los datos')

el valor de los nulos en la columna cancellation_month representa un 88.0% de los datos


Para estas dos columnas, el número de nulos es muy elevado, representa el 88% de la muestra por tanto, no incluimos estas dos columnas en nuestro análisis.

In [20]:
country = loyalty_history["Country"].unique()
print(f"el país origen de nuestros clientes es: {country}")
print("----------------------------------------")
Province = loyalty_history["Province"].unique()
print(f"las provincias de {country} son: {Province}")
print("----------------------------------------")
Gender = loyalty_history["Gender"].unique()
print(f'el género de los clientes en el DF es: {Gender}')
print("----------------------------------------")
education= loyalty_history["Education"].unique()
print(f'El nivel educativo de los clientes se distribuye en: {education}')
print("----------------------------------------")
tipo_suscrip = loyalty_history["Enrollment Type"].unique()
print(f"el tipo de suscripción de los clientes es: {tipo_suscrip}")
print("----------------------------------------")
tipo_tarj = loyalty_history["Loyalty Card"].unique()
print(f"el tipo de tarjeta de los clientes es: {tipo_tarj}")
print("----------------------------------------")

el país origen de nuestros clientes es: ['Canada']
----------------------------------------
las provincias de ['Canada'] son: ['Ontario' 'Alberta' 'British Columbia' 'Quebec' 'Yukon' 'New Brunswick'
 'Manitoba' 'Nova Scotia' 'Saskatchewan' 'Newfoundland'
 'Prince Edward Island']
----------------------------------------
el género de los clientes en el DF es: ['Female' 'Male']
----------------------------------------
El nivel educativo de los clientes se distribuye en: ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
----------------------------------------
el tipo de suscripción de los clientes es: ['Standard' '2018 Promotion']
----------------------------------------
el tipo de tarjeta de los clientes es: ['Star' 'Aurora' 'Nova']
----------------------------------------


In [21]:
#Renombramos columnas

lh_columns = loyalty_history.columns.str.lower().str.replace(' ', '_')

loyalty_history.columns= lh_columns

loyalty_history.sample(1)

,loyalty_number,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
12825,617108,Canada,Quebec,Quebec City,G1B 3L5,Male,College,NaN,Divorced,Star,4476.81,Standard,2017,1,2017.0,9.0


In [22]:
#Unimos ambos ficheros

flt_info = flight_activity.merge(loyalty_history,on="loyalty_number")
flt_info.head(5)

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
2,100018,2018,10,6,4,10,3110,311.0,385,31,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
3,100018,2017,4,4,0,4,924,92.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
4,100018,2017,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN


In [23]:
data_exploring(flt_info)

La forma es (403760, 25)
_______________
Las columnas son Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed',
       'country', 'province', 'city', 'postal_code', 'gender', 'education',
       'salary', 'marital_status', 'loyalty_card', 'clv', 'enrollment_type',
       'enrollment_year', 'enrollment_month', 'cancellation_year',
       'cancellation_month'],
      dtype='object')
_______________
Los nulos son:


,index,0
0,loyalty_number,0
1,year,0
2,month,0
3,flights_booked,0
4,flights_with_companions,0
5,total_flights,0
6,distance,0
7,points_accumulated,0
8,points_redeemed,0
9,dollar_cost_points_redeemed,0


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,loyalty_number,int64
1,year,int64
2,month,int64
3,flights_booked,int64
4,flights_with_companions,int64
5,total_flights,int64
6,distance,int64
7,points_accumulated,float64
8,points_redeemed,int64
9,dollar_cost_points_redeemed,int64


#### Gesión de nulos

In [24]:
# En la columna Salary, reemplazamos los nulos por 0

flt_info['salary'].fillna(0, inplace=True)

# Quitamos de nuestro DF las columnas  cancellation month & cancellation year ya que el valor de los nulos representa el 88% de la muestra

flt_info = flt_info.drop(["cancellation_month", "cancellation_year"], axis=1)

In [25]:
flt_info

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
2,100018,2018,10,6,4,10,3110,311.0,385,31,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
3,100018,2017,4,4,0,4,924,92.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
4,100018,2017,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403755,999891,2018,8,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403756,999891,2018,9,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403757,999891,2018,10,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403758,999891,2018,11,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5


In [26]:
data_exploring(flt_info)

La forma es (403760, 23)
_______________
Las columnas son Index(['loyalty_number', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed',
       'country', 'province', 'city', 'postal_code', 'gender', 'education',
       'salary', 'marital_status', 'loyalty_card', 'clv', 'enrollment_type',
       'enrollment_year', 'enrollment_month'],
      dtype='object')
_______________
Los nulos son:


,index,0
0,loyalty_number,0
1,year,0
2,month,0
3,flights_booked,0
4,flights_with_companions,0
5,total_flights,0
6,distance,0
7,points_accumulated,0
8,points_redeemed,0
9,dollar_cost_points_redeemed,0


_______________
Hay 0 duplicados
_______________
Los datos son de tipo:


,index,0
0,loyalty_number,int64
1,year,int64
2,month,int64
3,flights_booked,int64
4,flights_with_companions,int64
5,total_flights,int64
6,distance,int64
7,points_accumulated,float64
8,points_redeemed,int64
9,dollar_cost_points_redeemed,int64


#### Analizamos la coherencia de los datos

In [27]:
flt_book = flt_info.loc[flt_info["flights_booked"]==0,:]

flt_book

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month
4,100018,2017,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
17,100018,2018,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
18,100018,2018,6,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
20,100018,2018,8,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
25,100102,2017,2,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,M1R 4K3,Male,College,0.0,Single,Nova,2887.74,Standard,2013,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403755,999891,2018,8,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403756,999891,2018,9,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403757,999891,2018,10,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403758,999891,2018,11,0,0,0,0,0.0,0,0,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5


Filtramos todas aquellas reservas que son igual a 0 porque no nos aportan información, en estos casos, el número de vuelos fue 0

In [28]:
flt_info.drop(flt_info[flt_info["flights_booked"] == 0].index, inplace=True)

In [29]:
flt_info

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
2,100018,2018,10,6,4,10,3110,311.0,385,31,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
3,100018,2017,4,4,0,4,924,92.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
5,100018,2017,6,4,1,5,4330,433.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403717,999788,2017,6,5,3,8,4720,472.0,0,0,Canada,Ontario,London,M5B 3E4,Male,Bachelor,56345.0,Married,Nova,8409.07,Standard,2017,2
403718,999788,2017,7,7,0,7,3346,334.0,0,0,Canada,Ontario,London,M5B 3E4,Male,Bachelor,56345.0,Married,Nova,8409.07,Standard,2017,2
403719,999788,2017,8,11,0,11,3575,357.0,0,0,Canada,Ontario,London,M5B 3E4,Male,Bachelor,56345.0,Married,Nova,8409.07,Standard,2017,2
403730,999788,2017,10,4,3,7,868,86.0,361,29,Canada,Ontario,London,M5B 3E4,Male,Bachelor,56345.0,Married,Nova,8409.07,Standard,2017,2


#### Adecuamos el formato de las columnas

In [30]:
flt_info['loyalty_number'] = flt_info['loyalty_number'].astype(str)
flt_info['year'] = pd.to_datetime(flt_info['year'], format='%Y')
flt_info['month'] = pd.to_datetime(flt_info['month'], format='%M')
flt_info['points_accumulated'] = flt_info['points_accumulated'].astype(int)
flt_info['dollar_cost_points_redeemed'] = flt_info['dollar_cost_points_redeemed'].astype(float)
flt_info['enrollment_year'] = pd.to_datetime(flt_info['enrollment_year'], format='%Y')
flt_info['enrollment_month'] = pd.to_datetime(flt_info['enrollment_month'], format='%M')

In [31]:
flt_info.dtypes

loyalty_number                         object
year                           datetime64[ns]
month                          datetime64[ns]
flights_booked                          int64
flights_with_companions                 int64
total_flights                           int64
distance                                int64
points_accumulated                      int64
points_redeemed                         int64
dollar_cost_points_redeemed           float64
country                                object
province                               object
city                                   object
postal_code                            object
gender                                 object
education                              object
salary                                float64
marital_status                         object
loyalty_card                           object
clv                                   float64
enrollment_type                        object
enrollment_year                dat

Datos estadísticos de variables numéricas

In [32]:
np.round(flt_info.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
flights_booked,207632.0,8.04,4.67,1.00,4.00,8.00,11.00,21.00
flights_with_companions,207632.0,2.02,2.54,0.00,0.00,1.00,4.00,11.00
total_flights,207632.0,10.05,5.81,1.00,6.00,10.00,14.00,32.00
distance,207632.0,2361.63,1135.84,90.00,1458.00,2298.00,3150.00,6293.00
points_accumulated,207632.0,241.59,116.12,9.00,149.00,235.00,323.00,676.00
points_redeemed,207632.0,59.97,170.31,0.00,0.00,0.00,0.00,876.00
dollar_cost_points_redeemed,207632.0,4.85,13.78,0.00,0.00,0.00,0.00,71.00
salary,207632.0,59296.38,45861.55,-58486.00,0.00,63840.00,83224.00,407228.00
clv,207632.0,7956.92,6751.28,1898.01,3974.86,5768.83,8937.12,83325.38


Datos estadísticos de variables categóricas

In [33]:
flt_info.describe(include = "object").T

,count,unique,top,freq
loyalty_number,207632,15236,965605,39
country,207632,1,Canada,207632
province,207632,11,Ontario,67035
city,207632,29,Toronto,41958
postal_code,207632,55,V6E 3D9,11149
gender,207632,2,Male,103903
education,207632,5,Bachelor,129303
marital_status,207632,3,Married,120890
loyalty_card,207632,3,Star,94683
enrollment_type,207632,2,Standard,198576


## Segunda fase

**Distibución de vuelos durante el año**

In [ ]:
flt_info['year'].unique

<bound method Series.unique of 0         2017
1         2017
2         2018
3         2017
5         2017
          ... 
403717    2017
403718    2017
403719    2017
403730    2017
403739    2017
Name: year, Length: 207632, dtype: int64>